In [ ]:
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
from dataclasses import dataclass
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import ttest_ind
import requests
import pandas as pd
import json


def get_uniswap_data(pool_id, number_of_days,url):
    url = url
    query = f"""
    {{
        poolDayDatas(where: {{ pool: "{pool_id}" }}, first: {number_of_days}, orderBy: date, orderDirection: desc) {{
            date
            tick
            sqrtPrice
            liquidity
            volumeUSD
            volumeToken0
            volumeToken1
            tvlUSD
            feesUSD
        }}
    }}
    """
    response = requests.post(url, json={'query': query})
    data = response.json()

    return data['data']['poolDayDatas']
number_of_days=1000

treatment_pool_id= "0x1c3140ab59d6caf9fa7459c6f83d4b52ba881d36"
treatment_url="https://api.thegraph.com/subgraphs/name/ianlapham/optimism-post-regenesis"

control_pool_id="0x68f5c0a2de713a54991e01858fd27a3832401849"
control_url="https://api.thegraph.com/subgraphs/name/ianlapham/optimism-post-regenesis"#"https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3"

# Define Incentive and sample periods

incentive_start_date = pd.to_datetime('2023-01-18')
incentive_end_date = pd.to_datetime('2023-02-08')

incentive_period = incentive_end_date - incentive_start_date

sample_start = incentive_start_date - incentive_period
sample_end = incentive_end_date + incentive_period

#sample_start = pd.to_datetime('2022-12-01')
#sample_end = pd.to_datetime('2023-03-08')

uniswap_data = get_uniswap_data(treatment_pool_id,number_of_days,treatment_url)
df = pd.DataFrame(uniswap_data)

#wsteth/eth 0.05% ETH mainnaet control pool data 
uniswap_data = get_uniswap_data(control_pool_id,number_of_days,control_url)
control_df = pd.DataFrame(uniswap_data)

#Datatypes formatting
df['volumeUSD'] = df['volumeUSD'].astype(float)
df['volumeToken0'] = df['volumeToken0'].astype(float)
df['volumeToken1'] = df['volumeToken1'].astype(float)
df['sqrtPrice'] = (df['sqrtPrice'].astype(float))/2**96
df['liquidity'] = df['liquidity'].astype(float)
df['tvlUSD'] = df['tvlUSD'].astype(float)
df['fees'] = df['feesUSD'].astype(float)
df['date'] = pd.to_datetime(df['date'], unit='s')


#Data formatting
control_df['volumeUSD'] = control_df['volumeUSD'].astype(float)
control_df['volumeToken0'] = control_df['volumeToken0'].astype(float)
control_df['volumeToken1'] = control_df['volumeToken1'].astype(float)
control_df['sqrtPrice'] = (control_df['sqrtPrice'].astype(float))/2**96
control_df['liquidity'] = control_df['liquidity'].astype(float)
control_df['tvlUSD'] = control_df['tvlUSD'].astype(float)
control_df['fees'] = control_df['feesUSD'].astype(float)
control_df['date'] = pd.to_datetime(control_df['date'], unit='s')
#df['fees']= df['fees'].rolling(window=7).mean()
#control_df['fees']= control_df['fees'].rolling(window=7).mean()
df.head(100)
control_df.head(100)

In [26]:
#Optimism uniswap pools data
import requests
import pandas as pd
import json
def get_uniswap_data(pool_id, number_of_days):
    url = "https://api.thegraph.com/subgraphs/name/ianlapham/optimism-post-regenesis"
    query = f"""
    {{
        poolDayDatas(where: {{ pool: "{pool_id}" }}, first: {number_of_days}, orderBy: date, orderDirection: desc) {{
            date
            tick
            sqrtPrice
            liquidity
            volumeUSD
            volumeToken0
            volumeToken1
            tvlUSD
        }}
    }}
    """
    
    response = requests.post(url, json={'query': query})
    data = response.json()

    return data['data']['poolDayDatas']


uniswap_data = get_uniswap_data(pool_id="0x4a5a2a152e985078e1a4aa9c3362c412b7dd0a86",number_of_days=1000)

df = pd.DataFrame(uniswap_data)

df['volumeUSD'] = df['volumeUSD'].astype(float)
df['volumeToken0'] = df['volumeToken0'].astype(float)
df['volumeToken1'] = df['volumeToken1'].astype(float)
df['sqrtPrice'] = (df['sqrtPrice'].astype(float))/2**96
df['liquidity'] = df['liquidity'].astype(float)
df['tvlUSD'] = df['tvlUSD'].astype(float)
df['fees'] = df['volumeUSD'].astype(float)*0.0005


df['date'] = pd.to_datetime(df['date'], unit='s')


df.head(1000)


,date,tick,sqrtPrice,liquidity,volumeUSD,volumeToken0,volumeToken1
0,2023-06-12,201722,23995.632599,6.798816e+18,8.253202e+06,8.250674e+06,4745.092545
1,2023-06-11,201641,23897.965096,6.828219e+18,6.810535e+06,6.810376e+06,3878.028065
2,2023-06-10,201629,23884.034649,6.765875e+18,2.472667e+07,2.470985e+07,14036.418119
3,2023-06-09,201168,23339.613404,7.339233e+18,4.163894e+06,4.161772e+06,2262.260933
4,2023-06-08,201098,23258.873632,7.333534e+18,3.106909e+06,3.109436e+06,1684.098652
5,2023-06-07,201210,23388.644801,6.026796e+18,1.186461e+07,1.185596e+07,6379.380910
6,2023-06-06,200910,23041.144044,8.563993e+18,1.381841e+07,1.383174e+07,7473.726165
7,2023-06-05,201305,23500.42238,6.268972e+18,1.466255e+07,1.464906e+07,7995.847210
8,2023-06-04,200866,22989.808614,6.964187e+18,2.954634e+06,2.954968e+06,1555.580331
9,2023-06-03,200885,23011.706768,6.989561e+18,1.458666e+06,1.456384e+06,769.459989


In [36]:
#Optimism uniswap pools data

def get_uniswap_data():
    url = "https://api.thegraph.com/subgraphs/name/ianlapham/optimism-post-regenesis"
    query = """
    {
        poolDayDatas(where: { pool:"0x4a5a2a152e985078e1a4aa9c3362c412b7dd0a86"}, first: 1000, orderBy: date, orderDirection: desc) {
            date
            tick
            sqrtPrice
            liquidity
            volumeUSD
            volumeToken0
            volumeToken1
            
        }
    }
    """
    
    response = requests.post(url, json={'query': query})
    data = response.json()

    return data['data']['poolDayDatas']

uniswap_data = get_uniswap_data()

df = pd.DataFrame(uniswap_data)

df['volumeUSD'] = df['volumeUSD'].astype(float)
df['volumeToken0'] = df['volumeToken0'].astype(float)
df['volumeToken1'] = df['volumeToken1'].astype(float)
df['sqrtPrice'] = (df['sqrtPrice'].astype(float))/2**96
df['liquidity'] = df['liquidity'].astype(float)


df['date'] = pd.to_datetime(df['date'], unit='s')


df.head(10)


,date,tick,sqrtPrice,liquidity,volumeUSD,volumeToken0,volumeToken1
0,2023-06-12,1181,1.060869,4.130256e+21,19354.841948,9.844711,11.078964
1,2023-06-11,1191,1.061386,1.443869e+22,46891.232036,23.698822,26.654802
2,2023-06-10,1191,1.061404,1.429466e+22,52240.407153,26.374317,29.716535
3,2023-06-09,1193,1.061466,1.387562e+22,9626.238027,4.630441,5.216667
4,2023-06-08,1194,1.061535,1.389201e+22,7729.821283,3.722763,4.194236
5,2023-06-07,1199,1.061786,1.389201e+22,25161.614962,11.985903,13.512084
6,2023-06-06,1198,1.061772,5.011252e+22,68249.266747,33.228774,37.466207
7,2023-06-05,1197,1.061699,5.011252e+22,112668.952672,55.005493,61.989127
8,2023-06-04,1193,1.061498,5.011252e+22,30538.901480,14.233430,16.039675
9,2023-06-03,1190,1.061342,5.011252e+22,93591.259503,43.639858,49.178722


In [15]:
url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'

headers = {'Content-Type': 'application/json'}

# Replace 'your_pool_id' with the actual pool ID.
pool_id = '0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640'

query = f"""
{{
  pool(id: "{pool_id}") {{
    token0 {{
      id
      symbol
      name
      derivedETH
    }}
    token1 {{
      id
      symbol
      name
      derivedETH
    }}
    token0Price
    token1Price
    volumeToken0
    volumeToken1
    volumeUSD
    totalValueLockedToken0
    totalValueLockedToken1
    totalValueLockedUSD
  }}
}}
"""

response = requests.post(url, headers=headers, data=json.dumps({'query': query}))

# If the response status code is 200, the request was successful.
if response.status_code == 200:
    data = response.json()

    # Convert the pool data to a DataFrame:
    pool_df = pd.DataFrame([data['data']['pool']])
else:
    print(f"Request failed with status code {response.status_code}")
pool_df.head(5)

,token0,token1,token0Price,token1Price,volumeToken0,volumeToken1,volumeUSD,totalValueLockedToken0,totalValueLockedToken1,totalValueLockedUSD
0,{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606...,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,1737.393331493964895336372526812073,0.0005755749040087032559140738593494824,379302170831.743976,182377421.298955681974310442,379327901633.0362930489409256996493,184673007.229586,145030.849610560437512346,436119772.7795287152430958025870409


In [21]:
import requests
import json
import pandas as pd
from datetime import datetime

url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'

headers = {'Content-Type': 'application/json'}

# Replace 'your_pool_id' with the actual pool ID.
pool_id = '0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640'

query = f"""
{{
  poolDayDatas(orderBy: date, orderDirection: desc, first: 180, where: {{ pool: "{pool_id}" }}) {{
    date
    volumeUSD
    feesUSD
    sqrtPrice
    liquidity
    tick
    tvlUSD
  }}
}}
"""

response = requests.post(url, headers=headers, data=json.dumps({'query': query}))

# If the response status code is 200, the request was successful.
if response.status_code == 200:
    data = response.json()

    # Convert the pool day data to a DataFrame:
    pool_day_data_df = pd.DataFrame(data['data']['poolDayDatas'])

    # Convert timestamp to date
    pool_day_data_df['date'] = pd.to_datetime(pool_day_data_df['date'], unit='s')
else:
    print(f"Request failed with status code {response.status_code}")
pool_day_data_df.head(10)


,date,volumeUSD,feesUSD,sqrtPrice,liquidity,tick,tvlUSD
0,2023-06-12,167592169.3403100677651225925606744,83796.08467015503388256129628033753,1901183776490976514619827496118134,19651764774985396443,201723,436119591.8767804112883294475169089
1,2023-06-11,132187582.615282051224046020468309,66093.79130764102561202301023415535,1892467447785988312348709132151822,20751413951425536154,201631,438197749.4187395952698143620929341
2,2023-06-10,518252141.3683635397485859347462543,259126.0706841817698742929673731254,1892274321057076111059977999995079,20549014887474495402,201629,429806352.116992919428461758863794
3,2023-06-09,128995584.504335468797345300942891,64497.7922521677343986726504714457,1847128198475778595058358956571186,37179651669688486499,201146,464380331.4227017670049058712520517
4,2023-06-08,141813398.3952542823382166580337944,70906.69919762714116910832901689779,1843851015434337187358766175640004,36841047401471116733,201110,464096253.4213476613449103336582412
5,2023-06-07,296593710.5895915459137951137117678,148296.8552947957729568975568558833,1850688995940988482203065776820182,29693557278789472840,201184,461090868.7205308764683185386791982
6,2023-06-06,334008446.7298931010960051857946965,167004.223364946550548002592897347,1825314720710267389830507922432722,37400896360585223108,200908,467794400.5588222304925438390828037
7,2023-06-05,379610992.8623894171816656167820908,189805.4964311947085908328083910439,1861676730205864180889814534885799,28033452399493981388,201303,453515199.6093142909762696241271969
8,2023-06-04,108605785.1546696054036326982339876,54302.89257733480270181634911699378,1822411706917606102626254286955217,30689347858501364041,200876,461288982.1991788407190695459187013
9,2023-06-03,96205000.04028690040701419021683051,48102.50002014345020350709510841546,1821573671205129016419757689620020,32155425325673028539,200867,460179743.9158597663122029296562022
